# Ajuste de curva aos dados observacionais (H-alpha + HI) em NGC 4698

In [1]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
import plotly.graph_objs as go
from numpy import arange
from scipy.optimize import curve_fit
from matplotlib import pyplot

$$(M_{3.6\,\mu m} - M_{4.5\,\mu m})_{Bojo}=(M_{3.6\,\mu m} - M_{4.5\,\mu m})_{NGC\,4698}$$
$$(10.8527-M_{4.5\,\mu m})_{Bojo}=[-21.693-(-21.205)]_{NGC\,4698}$$
$$(10.8527-M_{4.5\,\mu m})_{Bojo}=(-0.488)_{NGC\,4698}$$
$$(M_{4.5\,\mu m})_{Bojo}=11.3407$$

In [2]:
# ra = np.array([0.2533,0.53,0.81,1.5,2.45,3.75,4.65,6.12])

# ve = np.array([22.18,91.06,148.8,150.3,182.98,200.8,208.5,223.8])

In [3]:
# Gás se aproximando

radius = np.array([0.2533,0.53,0.81,1.5,2.45,3.75,4.65,   6.47,  7.88143963,
        8.72587959,  9.57031955, 10.41475951, 11.25919947, 12.10363943,
       12.94807939, 13.79251935, 14.63695931, 15.48139927, 16.32583923,
       17.17027919, 18.01471915, 18.85915911, 19.70359907, 20.54803903,
       21.39247899, 22.23691895, 23.08135891, 23.92579886, 24.77023882,
       25.61467878, 26.45911874, 27.3035587 , 28.14799866, 28.99243862,
       29.83687858, 30.68131854, 31.5257585 , 32.37019846, 33.21463842,
       34.05907838, 34.90351834, 35.7479583 , 36.59239826, 37.43683822,
       38.28127818, 39.12571814, 39.9701581 ])

velocity = np.array([  22.18,91.06,148.8,150.3,182.98,200.8,208.5,207,
                     180.16201903, 187.35847012, 191.36304112,
       195.13639669, 196.37216136, 198.32158546, 196.31266947,
       201.42751252, 201.49357408, 201.36802063, 196.79505263,
       196.42502277, 196.01533165, 199.20709008, 200.72042276,
       199.80845702, 198.91632192, 198.73127657, 198.95598318,
       197.87880273, 196.26643855, 196.39199199, 197.01318957,
       196.23997743, 195.34784232, 196.82814423, 197.70044871,
       198.23569327, 197.61449569, 198.55949221, 199.63004217,
       202.22706413, 200.23797877, 199.82828765, 199.22035105,
       200.51554679, 202.14117195, 203.62141303, 209.03365473])

In [4]:
kpc = 3.086e+21 # conversão de [kpc] para [cm];
massa_solar = 1.9889999e+33 # conversão de [massa solar] para [g];
radius = radius * 3.086e+21 # passar de [kpc] para [cm], a fim de deixar as unidades no sistema cgs;
velocity = velocity * 1e5 # passar de [km s⁻¹] para [cm s⁻¹];
G = 6.67e-8 # constante gravitacional em [cm³ g⁻¹ s⁻²];
diametropixel = 0.085993765 # em [kpc]

In [5]:
# scale_radius_halo = (420.11903*2)*diametropixel # quantidade de pixels que correspondem ao raio halo multiplicada por 2,
#                                         # vezes o diâmetro físico de cada pixel
# print('O raio do halo é: ',scale_radius_halo,' kpc!') # em [kpc]

In [6]:
scale_disco_pixel = 34.9541
scale_length_disco = (scale_disco_pixel)*diametropixel # valor tirado de S4G: https://irsa.ipac.caltech.edu/data/SPITZER/S4G/galaxies/NGC4698/P4/NGC4698_bd.outgal

print('O scale-length do disco é: ',scale_length_disco,' kpc!') # em [kpc]

O scale-length do disco é:  3.0058346611865  kpc!


In [7]:
scale_length_bojo = (6.8191/1.7)*diametropixel # valor tirado de S4G: https://irsa.ipac.caltech.edu/data/SPITZER/S4G/galaxies/NGC4698/P4/NGC4698_bd.outgal
                                                # tá dividido por 1.8153 porque o paper de Hernquist mostra essa relação entre o raio efetivo e o lengh-scale.

print('O raio do bojo é: ',scale_length_bojo,' kpc!') # em [kpc]

O raio do bojo é:  0.3449412252420588  kpc!


In [18]:
def fit_curve(radius,M_h,R_h):
    G = 6.67e-8
    a = scale_length_disco * kpc
    R = radius
    R_b = scale_length_bojo * kpc
    M_b = 20470014091.042 * massa_solar
    M_d = 23090731299.643 * massa_solar
    
    comp1 = ((G*M_b*R)/(R+R_b)**2)
    comp2 = ((G*M_d*(R**2))/(((R**2)+(a**2))**(3/2)))
    comp3 = (R*((2*G*M_h*(R+R_h)*np.log((R+R_h)/R_h)-2*G*M_h*R)/((R**2)*np.log(4)*R_h + 
                                                    (R**3)*np.log(4)-(R**2)*R_h-R**3)))
    
    return np.sqrt(comp1 + comp2 + comp3)

In [19]:
a = scale_length_disco * kpc
R = radius
R_b = scale_length_bojo * kpc
M_b = 20470014091.042 * massa_solar
M_d = 23090731299.643 * massa_solar

popt, pcov = curve_fit(fit_curve, radius, velocity, p0=[2e11*massa_solar,20*kpc])#, 1.1*kpc, 35*kpc])
#                                                     bounds=((1e8*massa_solar, 1e10*massa_solar,
#                                                              1e11*massa_solar, 0.2*kpc, 5*kpc),
#                                                             (3e10*massa_solar, 3e11*massa_solar,
#                                                              5e15*massa_solar, 4*kpc, 5000*kpc)))
M_h,R_h = popt

In [20]:
print('Massa do Bojo (log):   ',round(np.log10(M_b/massa_solar),3))
print('Massa do Disco (log):  ',round(np.log10(M_d/massa_solar),3))
print('"Massa" do Halo (log): ',round(np.log10(M_h/massa_solar),3))
print('Scale lenght do Bojo:  ',round((R_b/kpc),3), 'kpc')
print('Scale radius do Halo:  ',round((R_h/kpc),2), 'kpc')
print('Scale lenght do Disco: ',round((a/kpc),3), 'kpc')

Massa do Bojo (log):    10.311
Massa do Disco (log):   10.363
"Massa" do Halo (log):  11.437
Scale lenght do Bojo:   0.345 kpc
Scale radius do Halo:   33.32 kpc
Scale lenght do Disco:  3.006 kpc


Qual a diferença entre effective radius, scale radius e scale lenght?

In [21]:
# Bojo de Hernquist
bojo = np.sqrt((G*M_b*R)/(R+R_b)**2)

# Disco de Kuzmin
disk = np.sqrt((G*M_d*(R**2))/(((R**2)+(a**2))**(3/2)))

# Halo de NFW
halo = np.sqrt(R*((2*G*M_h*(R+R_h)*np.log((R+R_h)/R_h)-2*G*M_h*R)/((R**2)*np.log(4)*R_h + 
                                                    (R**3)*np.log(4)-(R**2)*R_h-R**3)))

In [22]:
velocidade_mw = []
raios_mw = []
for line in open('/home/natanael/Área de Trabalho/Iniciação Científica/Milky Way rotation curve.txt', 'r'):
    values = [float(s) for s in line.split()]
    velocidade_mw.append(values[1])
    raios_mw.append(values[0])

In [23]:
fig = go.Figure()

# fig.add_trace(go.Scatter(x = raios,y = velocidade, line=dict(color='red', width=0.7, dash='solid'),
#                    mode = 'lines',
#                    marker = dict(symbol = 'circle-dot', size=3, color = 'red'),
#                    name = 'Milky Way (Sofue 2020)'))

fig.add_trace(go.Scatter(x = radius[0:8]/kpc,y = velocity[0:8]/1e5, line=dict(color='black', width=0.7, dash='solid'),
                   mode = 'markers',
                   marker = dict(symbol = 'diamond', size=3, color = 'red'),
                   name = 'H-alpha data (Rubin+99)'))

fig.add_trace(go.Scatter(x = radius[8:200]/kpc,y = velocity[8:200]/1e5, line=dict(color='black', width=0.7, dash='solid'),
                   mode = 'markers',
                   marker = dict(symbol = 'square', size=3, color = 'black'),
                   name = 'HI data'))

fig.add_trace(go.Scatter(x = radius/kpc, y = fit_curve(R, *popt)/1e5,line=dict(color='black', width=0.8),
                   mode = 'lines',
                   marker = dict(symbol = 'circle-dot', size=5, color = 'black'),
                   name = 'Best fit'))

fig.add_trace(go.Scatter(x = radius/kpc, y = bojo/1e5,line=dict(color='rgba(255,165,0, 1)', width=1, dash='dot'),
                   mode = 'lines',
                   marker = dict(symbol = 'circle-dot', size=3, color = 'rgba(255,165,0, 1)'),
                   name = 'Bulge (Hernquist)'))

fig.add_trace(go.Scatter(x = radius/kpc, y = disk/1e5,line=dict(color='royalblue', width=1, dash='dash'),
                   mode = 'lines',
                   marker = dict(symbol = 'circle-dot', size=3, color = 'rgba(8, 0, 246, 0.8)'),
                   name = 'Disk (Kuzmin)'))

fig.add_trace(go.Scatter(x = radius/kpc, y = halo/1e5,line=dict(color = 'rgba(153, 51, 230, 1.0)', width=1, 
                   dash='dashdot'), mode = 'lines',
                   marker = dict(symbol = 'circle-dot', size=3, color = 'rgba(153, 51, 153, 1.0)'),
                   name = 'Halo (NFW)'))

fig.add_vrect(x0="3.5", x1="4.5", 
              annotation_text="Milky Way scale-length", annotation_position="top left",
              fillcolor="green", opacity=0.2, line_width=0)

fig.add_vline(x=8, annotation_text="Sun position",annotation_position="bottom right",
              line_width=1.5, line_dash="dash", line_color="green",opacity=0.4)

fig.add_vline(x=scale_length_disco, annotation_text="",annotation_position="bottom right",
                  line_width=1.5, line_dash="dot", line_color="red",opacity=0.4)


fig.update_layout(legend=dict(yanchor="top",y=0.66,xanchor="right",x=0.999),
                              autosize=False,width=650,height=550,
                              margin=dict(l=50,r=50,b=100,t=100,pad=4),
                              paper_bgcolor='rgba(255,255,255,1)', plot_bgcolor='rgba(255,255,255,1)')

fig.update_layout(font = dict(family="tgheros", color='black', size=13),
                                      paper_bgcolor='rgba(255,255,255,1)', plot_bgcolor='rgba(255,255,255,1)')


fig.update_xaxes(showline=True, linewidth=0.005, linecolor='black',ticks="outside")
fig.update_yaxes(showline=True, linewidth=0.005, linecolor='black',ticks="outside")

fig.update_layout(title = dict(text = "Rotation curve decomposition - NGC 4698",x = .25,y=.9), 
                  xaxis = dict(range = [0,45], title = 'Radius     [kpc]'),
                  yaxis = dict(range = [0,270], title = 'Circular velocity     [km s⁻¹]'))

# with open("NGC4579 RC.html", "w") as f:
#     f.write(fig.to_html())

## Valores para a Milky Way

#### Lengh-scale do disco: 
Em torno de 3.5-4.5 kpc

4.8 kpc segundo Ken Freeman (https://ned.ipac.caltech.edu/level5/Sept15/Freeman/Freeman_contents.html)

Adicionar a componente de gás na curva de rotação a partir da massa que eu calculo com VIVA
##### 

$$\textrm{Massa cumulativa no modelo de Hernquist: } M_b(<R)= M_b \frac{R^2}{(R+A)^2} \qquad (Bojo)$$

In [43]:
bojo = bojo/1e5
massabojo = []
for i in range(len(bojo)):
    massabojo.append(M_b*(radius[i]**2/(radius[i]+scale_length_bojo)**2))
    
massabojo

[4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,
 4.0714855980081125e+43,


In [39]:
massa_do_bojo

array([10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814, 10.31111814, 10.31111814, 10.31111814,
       10.31111814, 10.31111814])

Talvez o problema seja pq eu deixei de considerar a massa do bojo como uma lista de massas que evolui com o raio e passei a usar só o valor absoluto final!